In [4]:
from a1_generate_instance import generateInstance
import numpy as np

a = generateInstance(days=3, size=20, cars=3, holding_cost="low")

In [ ]:
from gurobipy import Model, GRB, quicksum
def rFPPSolution(T, N, N_p, E, E_p, K, U, Q, r, I_0):
    '''
    T - time horizon
    N_p - set of customer
    N - set of customer + depot
    E_p - set of edges corresponding to customer
    E - set of edges corresponding to customer + depot
    K - set of vehicles
    U - capacity of inventory
    Q - capacity of vehicle
    r - daily consumption rate
    I_0 - initial inventory level
    '''
    
    m = Model("rFPP")
    I = m.addVars(N_p, T, vtype=GRB.CONTINUOUS, name="inventory_level")
    z = m.addVars(N, K, T, vtype=GRB.BINARY, name="is_visited_node")
    q = m.addVars(N_p, K, T, vtype=GRB.CONTINUOUS, name="quantity_delivered")
    y = m.addVars(E, K, T, vtype=GRB.BINARY, name="is_visited_edge")
    x = m.addVars(E, vtype=GRB.BINARY, name="masked_edge")

    # constraint about inventory level
    m.addConstrs((I[i,1] == I_0[i] + quicksum(q[i,k,1] for k in K) - r[i][1] for i in N_p), name="inventory_level_1")
    m.addConstrs((I[i,t] == I[i,t-1] + quicksum(q[i,k,t] for k in K) - r[i][t] for i in N_p for t in range(2,len(T)+1)), name="inventory_level_2")
    m.addConstrs((I[i,t] >= 0 for i in N_p for t in range(1,len(T)+1)), name="inventory_level_3")
    m.addConstrs((I[i,t] <= U[i] for i in N_p for t in range(1,len(T)+1)), name="inventory_level_4")

    # constraint about quantity delivered
    m.addConstrs((quicksum(q[i,k,t] for i in N_p) <= Q*z[0,k,t] for k in K for t in T), name="quantity_delivered_1")
    m.addConstrs((quicksum(q[i,k,1] for k in K) <= U[i]-I_0[i] for i in N_p), name="quantity_delivered_2")
    m.addConstrs((quicksum(q[i,k,t] for k in K) <= U[i]-I[i, t-1] for i in N_p for t in range(2,len(T)+1)), name="quantity_delivered_2")

    # constraint about routing
    m.addConstrs((quicksum(z[i,k,t] for k in K) <= 1 for i in N_p for t in T), name="routing_1")
    m.addConstrs((quicksum(y[i,j,k,t] + y[j,i,k,t] for i,j in E) == 2*z[i,k,t] for k in K for t in T for i in N_p), name="routing_2")

    # constraint about masking
    




    

